## finds top ROP

In [5]:
#filename = 'xa1606Pyr.csv'
filename = 'xa1609Oxi.csv'
factor = 0.95 # the fraction above which the script stops adding reactions
deadEnd = 0.1 # if consumption/totalROP is lower than this, then this species is a deadEnd
time = 0.000350 # make sure this exact time appears in the ROP file
minWidth = 0.25
maxWidth = 7.0
rtol = 1e-16

In [6]:
# read file
import csv, re, math
from operator import itemgetter
ROP = []
with open(filename) as f:
    csvReader = csv.reader(f)
    for row in csvReader:
        if 'Time (sec)' in row[0]:
            ROP.append(row) # ROP[0] is the headers row
        else:
            if time == float(row[0]):
                ROP.append(row) # ROP[1] is the data row

In [7]:
# get species
species = ['Time (sec)']
for col in ROP[0]:
    name = re.search('[^_]*', col).group(0)
    if name != species[-1]:
        species.append(name)

# extract data
output = {} # output dictionary. level 1: species; level 2: significantRates, normalizedRates, significantRxns, totalROP
col = 1 # skip the Time column
for spc in species[1:]:
    totalROP = ROP[1][col] # this is saved in the original sign (positive for net generation and vice versa);
    col += 1 # skip the ROP_total column, it includes production and consumption, we only care about consumption
    totalConsumption = 0
    rates = []
    rxns = []
    while re.search('[^_]*', ROP[0][col]).group(0) == spc:
        if float(ROP[1][col]) < 0: # only look at consumption rates
            totalConsumption -= float(ROP[1][col])# but report in positive terms for further processing
            rates.append(-float(ROP[1][col]))
            rxns.append(ROP[0][col])
        col += 1
        if col == len(ROP[0]):
            break
    combined = zip(rates,rxns)
    rated = sorted(combined, key=itemgetter(0),reverse=True)
    significantRates = []
    normalizedRates = [] # space saver; will be calculated later on
    significantRxns = []
    cummulativeRate = 0
    for rateRxnCuople in rated:
        cummulativeRate += rateRxnCuople[0] # always add at least the first one
        significantRates.append(rateRxnCuople[0])
        normalizedRates.append(-1/math.log(rateRxnCuople[0])) # normalized = -1/ln(rate) at first
        significantRxns.append(rateRxnCuople[1])
        if cummulativeRate / totalConsumption > factor: # check if we're above definded percentage
            break
    output[spc] = [significantRates, normalizedRates, significantRxns, float(totalROP), -totalConsumption]

# get overall min and max normalizedRates for final normalization
maxRate = 0.00001
minspc = ''
minRate = 1.0
for spc in species[1:]:
    for i in xrange(len(output[spc][1])):
        if output[spc][0][i] < maxRate * rtol:
            output[spc][1][i] = 0.0
        Nrate = output[spc][1][i]
        # for min rate make sure this isn't a deadEnd species
        if output[spc][3] > 0 and abs(output[spc][4]) / abs(output[spc][3]) > deadEnd and Nrate < minRate and Nrate:
            minRate = Nrate
            minspc = spc
        elif Nrate > maxRate:
            maxRate = Nrate

# finalize normalization
for spc in species[1:]:
    for i in xrange(len(output[spc][1])):
        if output[spc][1][i]:
            output[spc][1][i] = minWidth + (output[spc][1][i]-minRate)*(maxWidth-minWidth)/(maxRate-minRate)

In [43]:
spc = 'H(5)' # species to process

print "Species: {0}".format(spc)
print "Total significant reactions: {0}\n".format(len(output[spc][2]))

for i in xrange(len(output[spc][2])):
    print "\n{0}.  Width: {1:.3} for rxn: {2}".format(i+1, output[spc][1][i], output[spc][2][i])

print "\n\nConsumption ratio: {0:.3}".format(abs(output[spc][4]) / abs(output[spc][3]))
if output[spc][3] > 0 and abs(output[spc][4]) / abs(output[spc][3]) < deadEnd:
    print "\n{0} is a dead end species!".format(spc)

Species: H(5)
Total significant reactions: 2


1.  Width: 7.0 for rxn: H(5)_ROP_H(5)+O2(2)<=>O(6)+OH(3)_GasRxn#1 (mole/cm3-sec)

2.  Width: 5.25 for rxn: H(5)_ROP_H(5)+CH2O(36)<=>H2(7)+HCO(32)_GasRxn#430 (mole/cm3-sec)


Consumption ratio: 4.34


In [ ]:
# output['C2H4(24)']

In [ ]:
minspc

In [ ]:
minRate

In [ ]:
output[spc][3] > 0 and abs(output[spc][4]) / abs(output[spc][3])

In [ ]:
output['CH3OO(128)']

In [ ]:
abs(output['CH3OO(128)'][4]) / abs(output['CH3OO(128)'][3])

In [ ]:
output